In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import zipfile
import os

zip_file = list(uploaded.keys())[0]   # gets the uploaded zip file name
extract_to = "/content/dataset"

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("✅ Dataset extracted to:", extract_to)


✅ Dataset extracted to: /content/dataset


In [ ]:
import os
print(os.listdir("/content/dataset")[:20])


['faces95']


In [ ]:
INPUT_FOLDER = "/content/dataset"
OUTPUT_FOLDER = "/content/sorted_photos"


In [ ]:
!pip install opencv-python imutils scikit-learn


In [ ]:
INPUT_FOLDER = "/content/dataset"     # your unzipped dataset
OUTPUT_FOLDER = "/content/sorted_photos"

import os
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

print("Input:", INPUT_FOLDER)
print("Output:", OUTPUT_FOLDER)


Input: /content/dataset
Output: /content/sorted_photos


In [ ]:
import cv2
import os
import shutil
import numpy as np
from imutils import paths
from sklearn.cluster import KMeans

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def extract_face(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) == 0:
        return None
    (x, y, w, h) = faces[0]
    face = gray[y:y+h, x:x+w]
    return cv2.resize(face, (100, 100))

encodings = []
image_paths = list(paths.list_images(INPUT_FOLDER))
labels = []

print(f"📷 Found {len(image_paths)} images. Extracting faces...")

for image_path in image_paths:
    image = cv2.imread(image_path)
    face = extract_face(image)

    if face is None:
        noface_dir = os.path.join(OUTPUT_FOLDER, "NO_FACE_DETECTED")
        os.makedirs(noface_dir, exist_ok=True)
        shutil.copy(image_path, noface_dir)
        continue

    encodings.append(face.flatten())  # store flattened data
    labels.append(image_path)

encodings = np.array(encodings)

print("✅ Face extraction complete!")
print("🔍 Clustering faces (grouping)... Please wait...")

# Set number of clusters automatically based on dataset size (min 2, max 8)
k = min(8, max(2, len(encodings)//10))

kmeans = KMeans(n_clusters=k, random_state=42).fit(encodings)
clusters = kmeans.labels_

print("✅ Clustering completed!")
print(f"📌 Detected approximately {k} different people.")

# Create folders and sort images
for idx, img_path in enumerate(labels):
    cluster_id = clusters[idx]
    person_folder = os.path.join(OUTPUT_FOLDER, f"Person_{cluster_id+1}")
    os.makedirs(person_folder, exist_ok=True)
    shutil.copy(img_path, person_folder)

print("🎉 Sorting DONE!")
print(f"✔ Sorted images saved in: {OUTPUT_FOLDER}")


📷 Found 1440 images. Extracting faces...
✅ Face extraction complete!
🔍 Clustering faces (grouping)... Please wait...
✅ Clustering completed!
📌 Detected approximately 8 different people.
🎉 Sorting DONE!
✔ Sorted images saved in: /content/sorted_photos


In [ ]:
for folder in sorted(os.listdir(OUTPUT_FOLDER)):
    path = os.path.join(OUTPUT_FOLDER, folder)
    if os.path.isdir(path):
        print(f"{folder} → {len(os.listdir(path))} images")


NO_FACE_DETECTED → 68 images
Person_1 → 60 images
Person_2 → 33 images
Person_3 → 343 images
Person_4 → 207 images
Person_5 → 93 images
Person_6 → 189 images
Person_7 → 163 images
Person_8 → 284 images


In [ ]:
import shutil
shutil.make_archive("/content/sorted_photos", 'zip', OUTPUT_FOLDER)

from google.colab import files
files.download("/content/sorted_photos.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>